In [1]:
import datajoint as dj

In [2]:
import ephys
import matplotlib.pyplot as plt
%matplotlib inline

Connecting daveliu@mesoscale-activity.datajoint.io:3306


In [ ]:
def dropdbs():
   print('dropping databases')
   for a in range(3):
       for d in ['ingest', 'ccf', 'ephys', 'experiment', 'lab']:
           try:
               schema = dj.schema(dj.config['%s.database' % d], locals())
               schema.drop(force=True)
           except Exception as e:
               print('error dropping {d} in attempt {a}: {e}'
                     .format(d=d, a=a, e=str(e)))
               pass

In [ ]:
dj.ERD(ephys)

In [3]:
mouse=ephys.experiment.lab.Animal()
mouse.insert1((399752, '2017-8-1'))
mouse.insert1((123456, '2017-8-1'))
mouse.insert1((400483, '2017-8-1'))
mouse.insert1((400482, '2017-8-1'))
mouse

animal Janelia ANM ID (6 digits),dob
123456,2017-08-01
399752,2017-08-01
400482,2017-08-01
400483,2017-08-01


In [4]:
person=ephys.experiment.lab.Person()
person.insert1(('daveliu', 'Dave Liu'))
person

username,fullname
daveliu,Dave Liu


In [5]:
rig=ephys.experiment.lab.Rig()
rig.insert1(('TRig3', 'Training rig 3'))
rig.insert1(('TRig2', 'Training rig 2'))
rig.insert1(('TRig1', 'Training rig 1'))
rig.insert1(('RRig', 'Recording rig'))
rig

rig,rig_description
RRig,Recording rig
TRig1,Training rig 1
TRig2,Training rig 2
TRig3,Training rig 3


In [6]:
animalWaterRestriction=ephys.experiment.lab.AnimalWaterRestriction()
animalWaterRestriction.insert1((123456, 'tw5'))
animalWaterRestriction.insert1((399752, 'dl7'))
animalWaterRestriction.insert1((400483, 'dl8'))
animalWaterRestriction.insert1((400482, 'dl9'))
animalWaterRestriction

animal Janelia ANM ID (6 digits),water_restriction water restriction number
123456,tw5
399752,dl7
400482,dl9
400483,dl8


In [ ]:
dj.ERD(ephys.experiment)

In [7]:
import ingest
rigDataPath = ingest.RigDataPath()
rigDataFile = ingest.RigDataFile()
rigDataFileIngest = ingest.RigDataFileIngest()

In [8]:
#importedSessionFile.drop()
rigDataFile.populate()

In [ ]:
rigDataFileIngest.populate()

In [9]:
files= rigDataFile.fetch()

In [ ]:
len(files)

In [10]:
files

array([ ('RRig', 'b67\\TW_autoTrain\\Session Data\\b67_TW_autoTrain_20171107_091040.mat'),
       ('RRig', 'b67\\TW_autoTrain\\Session Data\\b67_TW_autoTrain_20171109_102128.mat'),
       ('RRig', 'b67\\TW_autoTrain\\Session Data\\b67_TW_autoTrain_20171113_105817.mat'),
       ...,
       ('TRig3', 'wr8\\TW_autoTrain\\Session Data\\wr8_TW_autoTrain_20171024_203425.mat'),
       ('TRig3', 'wr8\\TW_autoTrain\\Session Data\\wr8_TW_autoTrain_20171024_205803.mat'),
       ('TRig3', 'wr8\\TW_autoTrain\\Session Data\\wr8_TW_autoTrain_20171024_213159.mat')],
      dtype=[('rig', 'O'), ('rig_data_file', 'O')])

In [ ]:
allFiles=rigDataFile.fetch()

In [ ]:
allFiles[500]

In [ ]:
session=ephys.experiment.Session()
sessionList=session.fetch()

In [ ]:
sessionList

In [ ]:
trialNote = ephys.experiment.TrialNote()

In [ ]:
import numpy as np

In [ ]:
behaviorTrial = ephys.experiment.BehaviorTrial()

In [ ]:
outcome = (behaviorTrial & 'animal = 399752').fetch('outcome', order_by='session')
early_lick = (behaviorTrial & 'animal = 399752').fetch('early_lick', order_by='session')

In [ ]:
data = np.zeros(len(outcome))
data1 = np.zeros(len(early_lick))
data2 = np.zeros(len(outcome))

In [ ]:
data[outcome=='hit'] = 1
data1[early_lick=='early'] = 1
data2[outcome=='ignore'] = 1

In [ ]:
trial_note = (trialNote & 'animal = 123456').fetch('trial_note', order_by='session')
results = list(map(int, trial_note))
protDiff = np.diff(results)
protChange = np.where(protDiff != 0)

In [ ]:
def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values, weights, 'valid')
    return sma

In [ ]:
yMA = movingaverage(data,200)
plt.plot(yMA)
plt.xlabel('Trials')
plt.ylabel('Proportion Correct')
for i in range(0, len(protChange[0])):
    plt.axvline(protChange[0][i], linewidth=4, color='r')


In [ ]:
yMA = movingaverage(data1,200)
plt.plot(yMA)
plt.xlabel('Trials')
plt.ylabel('Early Licks')
for i in range(0, len(protChange[0])):
    plt.axvline(protChange[0][i], linewidth=4, color='r')


In [ ]:
yMA = movingaverage(data2,20)
plt.plot(yMA)
plt.xlabel('Trials')
plt.ylabel('Ignores')
for i in range(0, len(protChange[0])):
    plt.axvline(protChange[0][i], linewidth=4, color='r')
plt.savefig('')

In [ ]:
len(outcome)